## MBA em Ciência de Dados
# Redes Neurais e Arquiteturas Profundas

### <span style="color:darkred">Módulo 5 - Redes neurais auto-associativas e geradoras
</span>

#### <span style="color:darkred">**Parte 2: Autoencoders para Redução de Dimensionalidade**</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy.random import seed
from tensorflow.random import set_seed
from tensorflow import keras
from tensorflow.keras import layers

Outra aplicação de auto-encoders é seu uso para **redução de dimensionalidade não supervisionada**

Vamos utilizar a base de dados Boston Housing e a tarefa de regressão

Essa base de dados possui 13 atributos originalmente, vamos aprender uma redução desse espaço.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.boston_housing.load_data()

mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

x_test -= mean
x_test /= std
print(x_test.shape)

target_dimensions = 6

(102, 13)


### O autoencoder será denso, com:

* uma camada de 10 dimensões intermediária (para o encoder e o decoder)
* a camada de projeção no espaco do código

In [3]:
input_data = keras.layers.Input(shape=(13,))

# encoder
e1 = keras.layers.Dense(10, activation='tanh')(input_data)
z = keras.layers.Dense(target_dimensions, activation='tanh')(e1)

# decoder
d1 = keras.layers.Dense(10, activation='tanh')(z)
output = keras.layers.Dense(13, activation='tanh')(d1)

autoencoder = keras.models.Model(input_data, output)
autoencoder.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                140       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                70        
_________________________________________________________________
dense_3 (Dense)              (None, 13)                143       
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


In [4]:
seed(1)
set_seed(1)

epochs = 150
batch_size = 8

# definindo um decaimento para a taxa de aprendizado
def scheduler(epoch, lr):
  if epoch < 50:
    return lr
  else:
    return np.clip(lr * tf.math.exp(-0.01), 0.00001, 0.001)

callbacklr = tf.keras.callbacks.LearningRateScheduler(scheduler)

autoencoder.compile(loss='mse',
              optimizer=keras.optimizers.Adam(lr=0.002))

hist_ae = autoencoder.fit(x_train, x_train,  
                    callbacks=[callbacklr], batch_size=batch_size, epochs=epochs)

Epoch 1/150
51/51 [==============================] - 0s 2ms/step - loss: 0.9225
Epoch 2/150
51/51 [==============================] - 0s 2ms/step - loss: 0.6409
Epoch 3/150
51/51 [==============================] - 0s 2ms/step - loss: 0.5056
Epoch 4/150
51/51 [==============================] - 0s 2ms/step - loss: 0.4473
Epoch 5/150
51/51 [==============================] - 0s 2ms/step - loss: 0.4138
Epoch 6/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3929
Epoch 7/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3781
Epoch 8/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3673
Epoch 9/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3588
Epoch 10/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3515
Epoch 11/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3457
Epoch 12/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3401
Epoch 13/150
51/51 [=================

---

### Uso em regressor externo

Com o AE treinado, podemos utilizá-lo para obter representações para instâncias do treinamento e do teste

Vamos compara esse novo espaço aprendido com o original e com uma projeção PCA

In [5]:
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [6]:
code_model = keras.models.Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dense_1').output)
code_train = np.asarray(code_model.predict(x_train))
code_test  = np.asarray(code_model.predict(x_test))
print("Training data size = ", code_train.shape)
print("Testing data size = ", code_test.shape)

Training data size =  (404, 6)
Testing data size =  (102, 6)


Criando a projeção PCA para comparação

In [7]:
pca = PCA(n_components=target_dimensions)
pca.fit(x_train)
pca_train = pca.transform(x_train)
pca_test = pca.transform(x_test)

Treinando os regressores

In [8]:
print('Treinando Regressor com Código AE...')
clf_ae = Ridge()
clf_ae.fit(code_train, y_train)
code_pred = clf_ae.predict(code_test)

Treinando Regressor com Código AE...


In [9]:
print('Treinando Regressor com PCA...')
clf_pca = Ridge()
clf_pca.fit(pca_train, y_train)
pca_pred = clf_pca.predict(pca_test)

Treinando Regressor com PCA...


In [10]:
print('Treinando Regressor com Dados Originais...')
clf_ori = Ridge()
clf_ori.fit(x_train, y_train)
y_pred = clf_ori.predict(x_test)

Treinando Regressor com Dados Originais...


In [11]:
print('Calculando score...')
score_ae = mean_squared_error(code_pred, y_test)
score_ori = mean_squared_error(y_pred, y_test)
score_pca = mean_squared_error(pca_pred, y_test)
print('\nscore original: %.2f' % (score_ori))
print('\nscore PCA: %.2f' % (score_pca))
print('\nscore AE: %.2f ' % (score_ae))


Calculando score...

score original: 23.11

score PCA: 21.63

score AE: 20.46 
